In [1]:
from data_preprocessing import SO_Data_preprocess, Openso_Preprocess
from utils.tools import fiilna, Save_Part_Data, Plot_History_Trend
from Indicator_Construction import Foward_Indicator, Trend_Table, Historcial_Table,Validation_Indicator_Performance
import pandas as pd 
import numpy as np

In [2]:
ABC_class='A' # A or B or C
PG_class=['ECG','IPSG','IDSG'] #IIOT->4['ISG','IAG','ICWG','ESG'] EIOT->3 ['ECG', 'IPSG','IDSG']
Region_class='CN' # CN:China (CKH是HUB廠), TW:Taiwan, EU , US
start_date="2018-01-01"
end_date="2024-08-01"
full_date_range = pd.date_range(start=start_date,end=end_date,freq='MS').normalize()   
#根據上述條件建立sql
Constrain_query="ABC_INDICATOR.str.startswith(@ABC_class)  and PG in @PG_class and SHIP_PLANT.str.startswith(@Region_class) and  year_month>=@start_date and year_month<=@end_date"


In [3]:
SO_Data=SO_Data_preprocess()
Used_Data=SO_Data.query(Constrain_query).reset_index(drop=True)
Used_Data=Used_Data[['COMPANY_ID','year_month','PD','MODEL','PART_NO','ORDER_QTY']]
Used_Data.head()

,COMPANY_ID,year_month,PD,MODEL,PART_NO,ORDER_QTY
0,C400315,2019-05-01,Industrial Storage,SQF-S25 830,SQF-S25M8-256G-SAC,30
1,C210568,2018-12-01,Industrial Storage,SQF-S25 830,SQF-S25M8-256G-SAC,1
2,C190199,2019-12-01,Industrial Storage,SQF-S25 830,SQF-S25M8-256G-SAC,2
3,C403906,2020-07-01,Industrial Storage,SQF-S25 830,SQF-S25M8-256G-SAC,1
4,C400490,2022-05-01,Industrial Storage,SQF-S25 830,SQF-S25M8-256G-SAC,4


In [4]:
Used_Data.PD.unique()

array(['Industrial Storage', 'AIMB', 'EBC', 'ESBC', 'Industrial Wireless'],
      dtype=object)

In [5]:
Used_Data.groupby(['PD'])['ORDER_QTY'].sum().reset_index()

,PD,ORDER_QTY
0,AIMB,69434
1,EBC,18392
2,ESBC,17504
3,Industrial Storage,46400
4,Industrial Wireless,6


In [6]:
Openso_1=Openso_Preprocess(version_year=1).query(Constrain_query).reset_index(drop=True)
Openso_2=Openso_Preprocess(version_year=2).query(Constrain_query).reset_index(drop=True)
Openso_3=Openso_Preprocess(version_year=3).query(Constrain_query).reset_index(drop=True)

In [7]:
PD_name='AIMB' #Panel PC AIMB
PD_Material=(Used_Data.query('PD==@PD_name').groupby(['PART_NO'])['ORDER_QTY'].sum()).nlargest(50).reset_index()
PD_Material.PART_NO.unique()

array(['AIMB-505G2-00A1E', 'AIMB-215D-S6B2', 'AIMB-R431F-21A1E'],
      dtype=object)

In [8]:
# 檔案存取格式
if 'ECG' in PG_class:
    PG='EIOT'
else:
    PG='IIOT'

In [9]:
file_path = r'C:\Users\mt.yang\Desktop\SCM_Indicator\EIOT_AB_AIMB_EBC(B).csv' 

In [10]:
total_num,correct_num, part_number=0, 0, 0
total_up_num, total_down_num, total_tp_num, total_tn_num= 0,0,0,0
plot=False
# 料號日期條件
Material_sql=' PART_NO==@PART_NO_name '
date_column='year_month'

for PART_NO_name in PD_Material.PART_NO.unique():
    
    PART_NO_SO=Used_Data.query(Material_sql).groupby([date_column])['ORDER_QTY'].sum().reset_index()
    #open part
    PART_NO_Openso_1=Openso_1.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_2=Openso_2.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_3=Openso_3.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    '''
    填補so & openso
    '''
    PART_NO_SO=fiilna(PART_NO_SO, full_date_range , QTY_name= 'ORDER_QTY' )
    PART_NO_Openso_1=fiilna(PART_NO_Openso_1, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_2=fiilna(PART_NO_Openso_2, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_3=fiilna(PART_NO_Openso_3, full_date_range , QTY_name= 'OPEN_QTY' )
    #用於驗證兩個月後需求量基準值
    PART_NO_SO['2_month_avg'] = PART_NO_SO['ORDER_QTY'].rolling(window=2).mean()

    #計算兩種backlog變化率的計算
    Indicator= Foward_Indicator(PART_NO_Openso_1, PART_NO_Openso_2, PART_NO_Openso_3 )
    Trend_Indicator =Trend_Table(Indicator)
    Historical_info=Historcial_Table(PART_NO_SO)

    Evaluate_Table= Validation_Indicator_Performance(PART_NO_name, Trend_Indicator, Historical_info)

    Save_Part_Data(file_path, PART_NO_SO,Trend_Indicator, PG, Region_class, PD_name, PART_NO_name ,save_file=None)

    # 總數統計
    total_num+=Evaluate_Table['Total_num'].iloc[0]
    correct_num+=(Evaluate_Table['TP_num']+Evaluate_Table['TN_num']).iloc[0]
    part_number+=1 # 料號總數統計
    total_up_num+=Evaluate_Table['Upward_num'].iloc[0]
    total_down_num+=Evaluate_Table['Downward_num'].iloc[0]
    total_tp_num+=Evaluate_Table['TP_num'].iloc[0]
    total_tn_num+=Evaluate_Table['TN_num'].iloc[0]
    '''
    畫指標和so圖 
    '''
    if plot:
        Plot_History_Trend(PART_NO_name, PART_NO_Openso_1, PART_NO_Openso_2,  PART_NO_SO, Trend_Indicator )
        print(Evaluate_Table)

d:\SCM 指標\Indicator_Construction.py:76: RuntimeWarning: invalid value encountered in scalar divide
  tp_rate=(tp_num/pos_indicator)*100


In [11]:
print("Final result")
print('料號總數', part_number)
print('所有指標總數:',total_num)
print('向上指標總數', total_up_num)
print('向下指標總數', total_down_num)
print('TP rate', (total_tp_num/total_up_num)*100)
print('TN rate', (total_tn_num/total_down_num)*100)
print('Total 此PD勝率', (correct_num/total_num)*100)

Final result
料號總數 3
所有指標總數: 27
向上指標總數 15
向下指標總數 12
TP rate 60.0
TN rate 91.66666666666666
Total 此PD勝率 74.07407407407408
